# Solution for Text extraction from Image   

We have started to apply the General Google Pytesseract to an image for text extraction 
so we are not able to get particular table formats 

In [1]:
#importing libraries 
import cv2  #Computer Vision library 

import pandas as pd #For Structuring data 

from PIL import Image #Image feature extraction 

import pytesseract #OCR



%matplotlib inline
#The line above is necesary to show Matplotlib's plots inside a Jupyter Notebook


from matplotlib import pyplot as plt #plotting the image
from skimage.feature import daisy
from skimage import color,io,exposure

Convert all  images,if any images is in jpeg format to jpg format as everyother image is in jpg format

In [2]:
# Include tesseract executable in your path
pytesseract.pytesseract.tesseract_cmd = "C:\\Program Files\\Tesseract-OCR\\tesseract"


In [3]:
# Create an image object of PIL library
image = Image.open('C:\\Users\\XXXXXXX\\Desktop\\Solution For text extraction\\image_table.png')


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\XXXXXXX\\Desktop\\Solution For text extraction\\image_table.png'

In [ ]:
# pass image into pytesseract module
# pytesseract is trained in many languages
image_to_text = pytesseract.image_to_string(image, lang='eng')

Data is not in a particular order what we are looking so better we go image preprocessing techniques to have better results

In [ ]:
print(image_to_text) #trying to print table data 

Image Preprocessing for extracting the text data from image 

In [ ]:
#Loading the images and converting grayscale and Ostu's threshold
image = cv2.imread('image_table.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]


In [ ]:
plt.figure(figsize=(16,8))

plt.imshow(thresh)
plt.show()

In [ ]:
# Remove horizontal lines
horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (50,1))
detect_horizontal = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
cnts = cv2.findContours(detect_horizontal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    cv2.drawContours(thresh, [c], -1, (0,0,0), 2)


In [ ]:
# Remove vertical lines
vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,15))
detect_vertical = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, vertical_kernel, iterations=2)
cnts = cv2.findContours(detect_vertical, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    cv2.drawContours(thresh, [c], -1, (0,0,0), 3)

In [ ]:

# Dilate to connect text and remove dots
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10,1))
dilate = cv2.dilate(thresh, kernel, iterations=2)
cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    area = cv2.contourArea(c)
    if area < 500:
        cv2.drawContours(dilate, [c], -1, (0,0,0), -1)

# Bitwise-and to reconstruct image
result = cv2.bitwise_and(image, image, mask=dilate)
result[dilate==0] = (255,255,255)


Bitwise-and to reconstruct image

In [ ]:
plt.figure(figsize=(16,8))

plt.imshow(result)
plt.show()

Remove vertical lines

In [ ]:


plt.figure(figsize=(16,8))

plt.imshow(dilate)
plt.show()

Converting image in to text format, and using OCR processing

In [ ]:
# OCR
data = pytesseract.image_to_string(result, lang='eng',config='--psm 6')
print(data)
data = pd.DataFrame([x.split(';') for x in data.split('\n')])
data.to_csv('image_table.text',index=False,header=False)


In [ ]:
import skimage

import skimage.io

from skimage import data

from skimage import color

from skimage.util import view_as_blocks

In [ ]:
from skimage.feature import hog

In [ ]:
image_ = skimage.io.imread('image_table.png')

plt.figure(figsize=(16,8))

plt.imshow(image_,cmap = 'gray')

In [ ]:
fd, hog_image = hog(image_,pixels_per_cell =(52,52),
                    block_norm='L2-Hys',
                   visualize=True,
                   multichannel= True)

In [ ]:
fig,axes = plt.subplots(1,2,figsize =(12,8), sharex=True,sharey=True)

ax = axes.ravel()

ax[0].set_title('Orginal')

ax[0].imshow(image_,cmap = "gray")

hog_image_rescaled = exposure.rescale_intensity(hog_image,in_range=(0,10))

ax[1].set_title('inverted')
ax[1].imshow(hog_image_rescaled,cmap = "gray")


fig.tight_layout()
plt.figure(figsize=(10,13))
plt.show()

In [ ]:
# OCR
data = pytesseract.image_to_string(result, lang='eng',config='--psm 6')
print(data)
data = pd.DataFrame([x.split(';') for x in data.split('\n')])
data.to_csv('image_table.csv',index=False,header=False)
